In [25]:
import sys
import os
# caution: path[0] is reserved for script path (or '' in REPL).
sys.path.insert(1, os.path.abspath('./../src'))

import legacy_data_loader
import datetime
import matplotlib.pyplot as plt
import pandas as pd
import enum
import numpy as np


%matplotlib qt

In [26]:
REPT = legacy_data_loader.load_compressed_rept_data(satellite = "a",
                                                    start = datetime.datetime(year = 2012, month = 9, day = 1),
                                                    end = datetime.datetime(year = 2014, month = 1, day = 1))


Loading REPT data between: 2012-09-01 00:00:00 and 2014-01-01 00:00:00.
Loading : REPT_201209_A.npz
Loading : REPT_201210_A.npz
Loading : REPT_201211_A.npz
Loading : REPT_201212_A.npz
Loading : REPT_201301_A.npz
Loading : REPT_201302_A.npz
Loading : REPT_201303_A.npz
Loading : REPT_201304_A.npz
Loading : REPT_201305_A.npz
Loading : REPT_201306_A.npz
Loading : REPT_201307_A.npz
Loading : REPT_201308_A.npz
Loading : REPT_201309_A.npz
Loading : REPT_201310_A.npz
Loading : REPT_201311_A.npz
Loading : REPT_201312_A.npz
Loading : REPT_201401_A.npz


In [27]:
E0 = np.array(pd.Series(REPT["FESA"][:, 0], index = REPT["EPOCH"]).rolling("18h").mean())
E1 = np.array(pd.Series(REPT["FESA"][:, 1], index = REPT["EPOCH"]).rolling("18h").mean())
E2 = np.array(pd.Series(REPT["FESA"][:, 2], index = REPT["EPOCH"]).rolling("18h").mean())
EPOCH = REPT["EPOCH"]

all_finite = np.isfinite(E0) & np.isfinite(E1) & np.isfinite(E2)

E0 = E0[all_finite]
E1 = E1[all_finite]
E2 = E2[all_finite]
EPOCH = EPOCH[all_finite]

In [28]:
class state_machine(enum.Enum):
    
    idle = 0
    finding_maximum = 1
    finding_minimum = 2

In [ ]:


def difference(y0, y1, comparator = "exact"):
    
    if comparator == "exact":
        
        return y0 - y1
    
    if comparator == "log":
        
        return np.log10(y0) - np.log10(y1)
    
    if comparator == "percent":
        
        return (y0 - y1) / y0

def find_minimums_and_maximums(xp, fp, tol = 0.1, comparator = "exact"):
        
    minimums = []
    maximums = []
    
    state = state_machine(0)
    
    mem = 0
        
    for i, y in enumerate(fp):
        
        
        if state == state_machine.idle:
            
            if difference(y, fp[mem], comparator = comparator) > tol:
                
                state = state_machine.finding_maximum
                minimums.append(xp[mem])
            
            if difference(y, fp[mem], comparator = comparator) < -1 * tol :
                
                state = state_machine.finding_minimum
                maximums.append(xp[mem])
                        
        
        elif state == state_machine.finding_minimum:
            
            if difference(y, fp[mem], comparator = comparator) < 0:
                
                mem = i
                continue
            
            if difference(y, fp[mem], comparator = comparator) > tol:
                
                minimums.append(xp[mem])
                state = state_machine.finding_maximum
        
        elif state == state_machine.finding_maximum:
                        
            if difference(y, fp[mem], comparator = comparator) > 0:
                
                mem = i
                continue    
            
            if difference(y, fp[mem], comparator = comparator) < (-1 * tol):
            
                maximums.append(xp[mem])
                state = state_machine.finding_minimum
            
    return minimums, maximums

In [ ]:
minimums, maximums = find_minimums_and_maximums(EPOCH, E0, tol=0.3, comparator = "log")

plt.plot(EPOCH, np.log10(E0))
#plt.semilogy(EPOCH, E1)
#plt.semilogy(EPOCH, E2)


for minimum in minimums:
    plt.vlines(x = minimum, ymin = 1, ymax = 8, color="red", linewidth = 2, label="Minimum")
    
for maximum in maximums:
    plt.vlines(x = maximum, ymin = 1, ymax = 8, color="green", linewidth = 2, label="Maximum")

plt.ylim(3, 7)
plt.ylabel("Logged Flux Orders of Magnitude")
plt.xlabel("Time")
plt.legend()

from collections import OrderedDict

handles, labels = plt.gca().get_legend_handles_labels()
by_label = OrderedDict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

plt.show()

In [ ]:
print(minimums, maximums)


[datetime.datetime(2012, 9, 1, 11, 38, 18, 370000), datetime.datetime(2012, 9, 3, 16, 36, 47, 856000), datetime.datetime(2012, 9, 28, 0, 40, 21, 996000), datetime.datetime(2012, 10, 1, 11, 40, 23, 150000), datetime.datetime(2012, 10, 9, 2, 38, 5, 581000), datetime.datetime(2012, 10, 14, 5, 17, 55, 722000), datetime.datetime(2012, 11, 14, 9, 52, 47, 493000), datetime.datetime(2013, 1, 14, 9, 31, 8, 394000), datetime.datetime(2013, 1, 18, 9, 23, 46, 876000), datetime.datetime(2013, 1, 27, 3, 2, 41, 871000), datetime.datetime(2013, 2, 14, 11, 20, 8, 658000), datetime.datetime(2013, 2, 22, 7, 43, 6, 751000), datetime.datetime(2013, 3, 1, 18, 18, 5, 607000), datetime.datetime(2013, 3, 17, 16, 27, 12, 283000), datetime.datetime(2013, 3, 30, 5, 13, 51, 574000), datetime.datetime(2013, 4, 25, 6, 47, 18, 858000), datetime.datetime(2013, 5, 1, 19, 17, 40, 304000), datetime.datetime(2013, 5, 19, 5, 21, 1, 576000), datetime.datetime(2013, 5, 25, 17, 9, 34, 990000), datetime.datetime(2013, 6, 1, 20